In [12]:
from owslib.wms import WebMapService
bbox = (-79.8, 40.5, -71.8, 45.1) # approximate bounding box of new york.
factor = 111139/30 # change-in-latitude to meters, divided by 30 m data resolution
factor /= 2  # otherwise we exceed memory limits for the server.
size = (int(factor*(bbox[2]-bbox[0])), int(factor*(bbox[3]-bbox[1])))
wms = WebMapService('https://www.mrlc.gov/geoserver/mrlc_display/wms')
img = wms.getmap(   layers=['CONUS_Land_Cover'],
                    srs='EPSG:4326',
                    bbox=bbox, 
                    size=size,
                    format='image/geotiff',
                    timeout = 300) # 5 minute timeout; took 4 minutes to run for me.
out = open('test_image2.geotiff', 'wb')
out.write(img.read())
out.close()

In [8]:
import os
import geopandas as gp
import pandas as pd
import rasterio
from shapely.geometry import Point

In [9]:

L2A = os.path.join('data','GEDI02_A_2019213195955_O03602_02_T01087_02_003_02_V002.csv') # arbitrary granule.
gediL2A = pd.read_csv(L2A)
gediL2A['geometry'] = gediL2A.apply(lambda row: Point(row['lon_lowestmode'], row['lat_lowestmode']), axis=1)
gediL2A = gp.GeoDataFrame(gediL2A)
src = rasterio.open('test_image2.geotiff')
coord_list = ([(x,y) for x,y in zip(gediL2A["geometry"].x, gediL2A["geometry"].y)])
gediL2A['land_cover'] = [x for x in src.sample(coord_list)]
gediL2A.head() # 'land_cover' column now contains RBG triples--just need to use map legend to convert to land cover type.

,Unnamed: 0,beam,channel,lat_lowestmode,lon_lowestmode,elev_lowestmode,delta_time,rh,land_cover_data/landsat_water_persistence,land_cover_data/landsat_treecover,land_cover_data/region_class,land_cover_data/urban_proportion,land_cover_data/urban_focal_window_size,shot_number,geometry,land_cover
0,116070,2,1,44.096791,-75.654799,152.25650,4.992713e+07,[-6.74 -6.44 -6.17 -5.95 -5.73 -5.54 -5.39 -5....,0,77.0,7,0,3,36020200200283918,POINT (-75.65480 44.09679),"[186, 216, 234]"
1,116080,2,1,44.099566,-75.648862,151.62357,4.992713e+07,[-6.81 -5.84 -5.09 -4.38 -3.89 -3.52 -3.25 -3....,0,99.0,7,0,3,36020200200283928,POINT (-75.64886 44.09957),"[186, 216, 234]"
2,116090,2,1,44.102291,-75.642843,154.21913,4.992713e+07,[-6.85 -6.17 -5.54 -4.79 -4.08 -3.52 -2.99 -2....,0,100.0,7,0,3,36020200200283938,POINT (-75.64284 44.10229),"[104, 170, 99]"
3,116100,2,1,44.104905,-75.636643,156.89682,4.992713e+07,[-2.39 -1.61 -0.89 -0.33 0.18 0.71 1.31 1....,0,95.0,7,0,3,36020200200283948,POINT (-75.63664 44.10491),"[186, 216, 234]"
4,116110,2,1,44.107490,-75.630387,156.47855,4.992713e+07,[-5.5 -4.68 -4.11 -3.67 -3.25 -2.92 -2.62 -2....,0,98.0,7,0,3,36020200200283958,POINT (-75.63039 44.10749),"[186, 216, 234]"
